# 0. Try approaches to find an effective way to merge datasets

In [5]:
# a trial case to load data into Python
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED

#file_path = ".././studies/TN_01/TrialNet_01_Data/TN01_ADVERSEEVENTS.CSV"
file_path = "/home/ec2-user/SageMaker/studies/TN_01/TrialNet_01_Data/TN01_nh01_screening.csv"

df1 = pd.read_csv(file_path)
df1.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/tmp/ipykernel_33999/705450557.py:8: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file_path)


,MaskID,Visit_Dt,Visit,Visit_Type,ScreeningDay,ScreeningMonth,ScreeningYear,Age,RelativeWithT1D,DegreeOfRelative_1st,...,clinicalcenternumber,ScreeningSiteName,ScreeningSiteNumber,WhichProtocolVersion,shipspecimencollectionkit,Specimencollectionkitshipmentd,Spring2019ConsentDateMonth,selecttestkittype,Spring2019ConsentDateDay,Spring2019ConsentDateYear
0,200000,06/29/2007,Screening,NaN,29.0,Jun,2007.0,20.0,Yes,0,...,6.0,Ponce School of Medicine,903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200003,06/29/2015,Screening,NaN,29.0,Jun,2015.0,20.0,Yes,1,...,NaN,NaN,3017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200004,08/11/2015,Screening,NaN,11.0,Aug,2015.0,44.0,Yes,1,...,NaN,NaN,3126.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200007,11/21/2016,Screening,NaN,21.0,Nov,2016.0,7.0,Yes,1,...,NaN,NaN,3126.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200010,10/21/2016,Screening,NaN,21.0,Oct,2016.0,34.0,Yes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234614 entries, 0 to 234613
Data columns (total 66 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   MaskID                            234614 non-null  int64  
 1   Visit_Dt                          234614 non-null  object 
 2   Visit                             234614 non-null  object 
 3   Visit_Type                        4674 non-null    object 
 4   ScreeningDay                      233579 non-null  float64
 5   ScreeningMonth                    233579 non-null  object 
 6   ScreeningYear                     233579 non-null  float64
 7   Age                               233440 non-null  float64
 8   RelativeWithT1D                   233554 non-null  object 
 9   DegreeOfRelative_1st              234614 non-null  int64  
 10  DegreeOfRelative_2ndor3rd         234614 non-null  int64  
 11  HaveDiabetes                      233488 non-null  o

In [20]:
df1.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234614 entries, 0 to 234613
Data columns (total 66 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   MaskID                            234614 non-null  int64  
 1   Visit_Dt                          234614 non-null  object 
 2   Visit                             234614 non-null  object 
 3   Visit_Type                        4674 non-null    object 
 4   ScreeningDay                      233579 non-null  float64
 5   ScreeningMonth                    233579 non-null  object 
 6   ScreeningYear                     233579 non-null  float64
 7   Age                               233440 non-null  float64
 8   RelativeWithT1D                   233554 non-null  object 
 9   DegreeOfRelative_1st              234614 non-null  int64  
 10  DegreeOfRelative_2ndor3rd         234614 non-null  int64  
 11  HaveDiabetes                      233488 non-null  o

# 1. Load merged datasets

## 1.1 TN_01: Merging into one

### 1.1.1 Data type transformation: string into category, float64 into float16

In [31]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# The following code is to merge the two datasets for TN_01, but will cause error or dead kernel. 
# Convert the 'ProtocolVersion' column to string in both DataFrames
file_path_1 = "TN_01_merged_data_1.csv"
file_path_2 = "TN_01_merged_data_23.csv"

merged_first_part = pd.read_csv(file_path_1)
merged_data_23 = pd.read_csv(file_path_2)

In [2]:
print(merged_first_part.info())
print()
print(merged_data_23.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519750 entries, 0 to 519749
Columns: 1563 entries, MaskID to SignedConsentFor2017Protocol
dtypes: float64(665), int64(1), object(897)
memory usage: 6.1+ GB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968807 entries, 0 to 1968806
Data columns (total 17 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   MaskID                         int64  
 1   Date_of_Registration           object 
 2   Status                         object 
 3   Date_of_Study_Start            object 
 4   Date_of_Draw                   object 
 5   Event_Title                    object 
 6   Spec_Name                      object 
 7   SampleMaskID                   object 
 8   Test_Name                      object 
 9   Result                         object 
 10  Result_Type                    object 
 11  Visit                          object 
 12  Date_at_Test_Results_Reported  object 
 13  Date_at_Ev

In [3]:
print(merged_first_part.memory_usage(deep=True).sum())
print()
print(merged_data_23.memory_usage(deep=True).sum())

17883089368

1802078130


##### By changing some columns from float64 to float 16. I checked the accuracy loss, which is minimal and negligible. 

In [32]:
import pandas as pd
import numpy as np

def transform_data_types(df):
    for column in df.columns:
        # Check for NaN values in the column
        has_nan = df[column].isnull().any()

        if df[column].dtype == 'float64':
            # Convert to int16 if all values are integers and there are no NaN values
            if not has_nan and all(df[column].dropna().apply(float.is_integer)):
                df[column] = df[column].astype('int16')
            else:
                # Convert to float16 otherwise
                df[column] = df[column].astype('float16')

        elif df[column].dtype == 'object':
            # Convert string columns to category
            if all(df[column].apply(lambda x: isinstance(x, str) or pd.isnull(x))):
                df[column] = df[column].astype('category')

    return df

In [33]:
transformed_merged_first_part = transform_data_types(merged_first_part)
transformed_merged_second_part = transform_data_types(merged_data_23)

print(transformed_merged_first_part.memory_usage(deep=True).sum())
print()
print(transformed_merged_second_part.memory_usage(deep=True).sum())

1311209169

245206343


In [5]:
print(transformed_merged_first_part.info())
print()
print(transformed_merged_second_part.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519750 entries, 0 to 519749
Columns: 1563 entries, MaskID to SignedConsentFor2017Protocol
dtypes: category(888), float16(665), int64(1), object(9)
memory usage: 1.1+ GB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968807 entries, 0 to 1968806
Data columns (total 17 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   MaskID                         int64   
 1   Date_of_Registration           category
 2   Status                         category
 3   Date_of_Study_Start            category
 4   Date_of_Draw                   category
 5   Event_Title                    category
 6   Spec_Name                      category
 7   SampleMaskID                   category
 8   Test_Name                      category
 9   Result                         object  
 10  Result_Type                    category
 11  Visit                          category
 12  Date_at_Test_Results_Report

### 1.2.2 Data merge into one

In [34]:
common_headers_123 = set.intersection(set(transformed_merged_first_part.columns), set(transformed_merged_second_part.columns))

# Merge the first two DataFrames on common headers using outer join
merged_data_all = pd.merge(transformed_merged_first_part, transformed_merged_second_part, on=list(common_headers_123), how='outer', 
                            suffixes=('_dup_x', '_dup_y'))

print(merged_data_all.info())
merged_data_all.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2923848 entries, 0 to 2923847
Columns: 1578 entries, MaskID to LabID
dtypes: category(900), float16(666), int64(1), object(11)
memory usage: 6.4+ GB
None


,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,_22Jul2009ConsentSignedDay,_22Jul2009ConsentSignedMonth,_22Jul2009ConsentSignedYear,...,Spec_Name,SampleMaskID,Test_Name,Result,Result_Type,Date_at_Test_Results_Reported,Date_at_Evaluation,Date_Received,Date_Shipped,LabID
0,200023,07/01/2015,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,Serum - Autoantibodies,S3827078,GAD65H,0,RPTD,07/20/2015,07/14/2015,07/14/2015,07/08/2015,547.0
1,200023,07/01/2015,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,Serum - Autoantibodies,S3827078,MIAA,0,RPTD,07/20/2015,07/14/2015,07/14/2015,07/08/2015,547.0
2,200023,07/01/2015,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,Serum - Autoantibodies,S3827078,IA-2H,0,RPTD,07/20/2015,07/14/2015,07/14/2015,07/08/2015,547.0
3,200071,01/24/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200071,01/14/2019,PRN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#merged_data_all.to_csv("TN_01_Merged_Data.csv", index = False)

#### The merged dataset is too large to be saved in the working directory. We are to turn columns with numerical values and large portion of missing values into sparse data structure.

In [36]:
summary_df = pd.read_csv('merged_data_summary.csv')
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1578 entries, 0 to 1577
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   header              1578 non-null   object 
 1   Missing Percentage  1578 non-null   float64
 2   Value Type          1578 non-null   object 
 3   Top 1               1577 non-null   object 
 4   Top 2               1192 non-null   object 
 5   Top 3               708 non-null    object 
 6   Top 4               580 non-null    object 
 7   Top 5               526 non-null    object 
 8   Top 1 Value Count   1577 non-null   float64
 9   Top 2 Value Count   1192 non-null   float64
 10  Top 3 Value Count   708 non-null    float64
 11  Top 4 Value Count   580 non-null    float64
 12  Top 5 Value Count   526 non-null    float64
dtypes: float64(6), object(7)
memory usage: 160.4+ KB


### 1.2.4 Numerical data structure transformation

In [37]:
# Filter based on Missing Percentage and Value Type
filtered_columns = summary_df[(summary_df['Missing Percentage'] > 85) & 
                             (summary_df['Value Type'].isin(['float16', 'int64']))]

# Get the list of headers from the filtered DataFrame
headers_with_missing_85 = filtered_columns['header'].tolist()

# Print the count of columns
print("Count of columns with missing Percentage > 85% and Value Type 'float16' or 'int64':", len(headers_with_missing_85))

Count of columns with missing Percentage > 85% and Value Type 'float16' or 'int64': 635


#### Tried many times and found that: 1) for columns with very high missing values, sparse data structure significantly save memory; 2) when missing value less than 85%, sparse data structure does have advantage over float16. 

In [38]:
print(merged_data_all.info())
print()
print(merged_data_all.memory_usage(deep=True).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2923848 entries, 0 to 2923847
Columns: 1578 entries, MaskID to LabID
dtypes: category(900), float16(666), int64(1), object(11)
memory usage: 6.4+ GB
None

7855246123


In [39]:
error_columns = []  # List to store columns that raised errors

for col in headers_with_missing_85:
    try:
        print("Before transformation of", col, ":", merged_data_all[col].memory_usage(deep=True), "bytes")
        
        # Check data type of the column
        col_dtype = merged_data_all[col].dtype
        print(col_dtype)
        if col_dtype == 'float16' or col_dtype == 'int64':
            # Convert numeric columns to default sparse type
            merged_data_all[col] = pd.to_numeric(merged_data_all[col], errors='coerce').astype('Sparse[float16]')
        # No need to replace the column in the original DataFrame
        # Just printing out the memory usage after transformation
        print("After transformation to Sparse of", col, ":", merged_data_all[col].memory_usage(deep=True), "bytes")
    except ValueError as e:
        print(f"Error converting column {col}: {e}")
        error_columns.append(col)

    print()

# Print out columns that had errors
if error_columns:
    print("Columns with errors:", error_columns)

Before transformation of Thiazolidinediones_dup22 : 5847824 bytes
float16
After transformation to Sparse of Thiazolidinediones_dup22 : 128 bytes

Before transformation of _2cTN18Exclusion_1aAbnormalGluc1 : 5847824 bytes
float16
After transformation to Sparse of _2cTN18Exclusion_1aAbnormalGluc1 : 134 bytes

Before transformation of participantnoteligib_5Forfemales : 5847824 bytes
float16
After transformation to Sparse of participantnoteligib_5Forfemales : 134 bytes

Before transformation of TN22ExclusionCriteri_Historyoftr : 5847824 bytes
float16
After transformation to Sparse of TN22ExclusionCriteri_Historyoftr : 134 bytes

Before transformation of TN22ExclusionCriteri_Historyofre : 5847824 bytes
float16
After transformation to Sparse of TN22ExclusionCriteri_Historyofre : 134 bytes

Before transformation of _2cTN20Exclusion_1DiagnosedwithD : 5847824 bytes
float16
After transformation to Sparse of _2cTN20Exclusion_1DiagnosedwithD : 134 bytes

Before transformation of _2cTN20Exclusion_8O

In [40]:
print(merged_data_all.info())
print()
print(merged_data_all.memory_usage(deep=True).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2923848 entries, 0 to 2923847
Columns: 1578 entries, MaskID to LabID
dtypes: Sparse[float16, nan](635), category(900), float16(31), int64(1), object(11)
memory usage: 3.0+ GB
None

4195149037


In [41]:
for column in merged_data_all.columns:
    if merged_data_all[column].dtype == 'object':
        merged_data_all[column] = merged_data_all[column].astype('category')

In [42]:
#merged_data_all = merged_data_all.drop('Unnamed: 0', axis=1)

In [43]:
print(merged_data_all.info())
print()
print(merged_data_all.memory_usage(deep=True).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2923848 entries, 0 to 2923847
Columns: 1578 entries, MaskID to LabID
dtypes: Sparse[float16, nan](635), category(911), float16(31), int64(1)
memory usage: 2.8 GB
None

3042099455


#### change MaskID from int64 to int32

In [44]:
test_col = merged_data_all['MaskID']
print(test_col.memory_usage(deep=True))
print(test_col.head(30))
print()
test_col_16 = test_col.astype('int16')
print(test_col_16.memory_usage(deep=True))
print(test_col_16.head(30))
print()
test_col_32 = test_col.astype('Int32')
print(test_col_32.memory_usage(deep=True))
print(test_col_32.head(30))
print()

23390912
0     200023
1     200023
2     200023
3     200071
4     200071
5     200071
6     200071
7     200074
8     200074
9     200074
10    200074
11    200074
12    200074
13    200082
14    200082
15    200082
16    200082
17    200082
18    200082
19    200082
20    200082
21    200082
22    200082
23    200082
24    200082
25    200082
26    200082
27    200082
28    200082
29    200082
Name: MaskID, dtype: int64

5847824
0     3415
1     3415
2     3415
3     3463
4     3463
5     3463
6     3463
7     3466
8     3466
9     3466
10    3466
11    3466
12    3466
13    3474
14    3474
15    3474
16    3474
17    3474
18    3474
19    3474
20    3474
21    3474
22    3474
23    3474
24    3474
25    3474
26    3474
27    3474
28    3474
29    3474
Name: MaskID, dtype: int16

14619368
0     200023
1     200023
2     200023
3     200071
4     200071
5     200071
6     200071
7     200074
8     200074
9     200074
10    200074
11    200074
12    200074
13    200082
14    200082
15 

In [45]:
merged_data_all['MaskID'] = merged_data_all['MaskID'].astype('Int32')

print(merged_data_all.info())
print()
print(merged_data_all.memory_usage(deep=True).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2923848 entries, 0 to 2923847
Columns: 1578 entries, MaskID to LabID
dtypes: Int32(1), Sparse[float16, nan](635), category(911), float16(31)
memory usage: 2.8 GB
None

3033327911


In [46]:
unique_values = merged_data_all['D5MeterReadingResult4_1'].unique()
print(unique_values)

[nan, 16.2, 11.8, 9.8, 155.0, 294.0, 20.2, 133.0]
Fill: nan
IntIndex
Indices: array([1, 2, 3, 4, 5, 6, 7], dtype=int32)



#### turn columns as Sparse[float64] to Sparse[float16]

In [47]:
# Iterate over each column and convert if it's of type Sparse[float64]
for column in merged_data_all.columns:
    if pd.api.types.is_sparse(merged_data_all[column].dtype):
        if merged_data_all[column].dtype.subtype == np.float64:
            merged_data_all[column] = merged_data_all[column].astype('Sparse[float16]')

print(merged_data_all.info())
print()
print(merged_data_all.memory_usage(deep=True).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2923848 entries, 0 to 2923847
Columns: 1578 entries, MaskID to LabID
dtypes: Int32(1), Sparse[float16, nan](635), category(911), float16(31)
memory usage: 2.8 GB
None

3033327911


#### turn columns as float16 to category

In [48]:
test_col = merged_data_all['Race_White']

print("memory used as float16: ",test_col.memory_usage(deep=True))
print()

# Convert to float32 or float64 before converting to category
test_col_float32 = test_col.astype('float32')
print("memory used as float32: ",test_col_float32.memory_usage(deep=True))
print()

test_col_category = test_col_float32.astype('category')
print("memory used as category: ", test_col_category.memory_usage(deep=True))


memory used as float16:  5847824

memory used as float32:  11695520

memory used as category:  2924076


In [49]:
# Iterate over each column and convert if it's of type float16
for column in merged_data_all.columns:
    if merged_data_all[column].dtype == np.float16:
        print("memory used as float16: ", merged_data_all[column].memory_usage(deep=True))
        #convert to float32 first
        merged_data_all[column] = merged_data_all[column].astype('float32')
        print("memory used as float32: ", merged_data_all[column].memory_usage(deep=True))
        # Convert the column to 'category'
        merged_data_all[column] = merged_data_all[column].astype('category')
        print("memory used as category: ", merged_data_all[column].memory_usage(deep=True))
        print()

# Display DataFrame info and memory usage
print(merged_data_all.info())
print()
print(merged_data_all.memory_usage(deep=True).sum())


memory used as float16:  5847824
memory used as float32:  11695520
memory used as category:  2924932

memory used as float16:  5847824
memory used as float32:  11695520
memory used as category:  2925808

memory used as float16:  5847824
memory used as float32:  11695520
memory used as category:  5855476

memory used as float16:  5847824
memory used as float32:  11695520
memory used as category:  2924076

memory used as float16:  5847824
memory used as float32:  11695520
memory used as category:  2924076

memory used as float16:  5847824
memory used as float32:  11695520
memory used as category:  2925860

memory used as float16:  5847824
memory used as float32:  11695520
memory used as category:  2924076

memory used as float16:  5847824
memory used as float32:  11695520
memory used as category:  2924076

memory used as float16:  5847824
memory used as float32:  11695520
memory used as category:  2924076

memory used as float16:  5847824
memory used as float32:  11695520
memory used as 

#### Turn SampleMaskID into integer

In [50]:
# SampleMaskID has the biggest memory
# To turn it into interger, we need to check whether all values are starting with 'S', and the rest of values are integer.
column = merged_data_all['SampleMaskID']

print("memory as category: ", column.memory_usage(deep=True))

test_col_obj = test_col.astype('object')
print("memory as object: ",test_col_obj.memory_usage(deep=True))

# Filter out missing values
non_missing_column = column[column.notna()]

# Check if values start with 'S'
starts_with_s = non_missing_column.str.startswith('S')

# Check if the rest part of the values are integers
rest_is_integer = non_missing_column.str.lstrip('S').apply(lambda x: x.isdigit())

# Combine the conditions
all_conditions_met = starts_with_s & rest_is_integer

# Check if all non-missing values in the column meet the conditions
all_values_meet_conditions = all_conditions_met.all()

print("Do all non-missing values start with 'S' and the rest part are integers?", all_values_meet_conditions)

memory as category:  77092158
memory as object:  93563264
Do all non-missing values start with 'S' and the rest part are integers? True


##### No.1 change in column values: remove 'S' for 'SampleMaskID'

In [51]:
# Strip 's' from the start of each string in the column
merged_data_all['SampleMaskID'] = merged_data_all['SampleMaskID'].str.lstrip('S')
print("original memory usage: ", merged_data_all['SampleMaskID'].memory_usage(deep=True))

original memory usage:  175059520


In [52]:
merged_data_all['SampleMaskID'].head(30)

0     3827078
1     3827078
2     3827078
3         NaN
4         NaN
5         NaN
6         NaN
7     9194466
8     9194466
9     9194466
10    9615167
11    9615167
12    9615167
13    2477315
14    2477315
15    2477315
16    9791970
17    9791970
18    9791970
19    4257250
20    4257250
21    4257250
22    2477315
23    2477315
24    2477315
25    9791970
26    9791970
27    9791970
28    4257250
29    4257250
Name: SampleMaskID, dtype: object

In [53]:
# SexOfRelativeWithT1D14_1 as category in the dataframe with only one value: Male.  
test_col = merged_data_all['SampleMaskID'] 

print("original memory usage: ", test_col.memory_usage(deep=True))
test_col_32 = test_col.astype('Int32')
print("memory usage as Int32: ", test_col_32.memory_usage(deep=True))
print(test_col_32.head(30))

test_col_16 = test_col.astype('Int16')
print("memory usage as Int16: ", test_col_16.memory_usage(deep=True))
print(test_col_16.head(30))

original memory usage:  175059520
memory usage as Int32:  14619368
0     3827078
1     3827078
2     3827078
3        <NA>
4        <NA>
5        <NA>
6        <NA>
7     9194466
8     9194466
9     9194466
10    9615167
11    9615167
12    9615167
13    2477315
14    2477315
15    2477315
16    9791970
17    9791970
18    9791970
19    4257250
20    4257250
21    4257250
22    2477315
23    2477315
24    2477315
25    9791970
26    9791970
27    9791970
28    4257250
29    4257250
Name: SampleMaskID, dtype: Int32
memory usage as Int16:  8771672
0      25990
1      25990
2      25990
3       <NA>
4       <NA>
5       <NA>
6       <NA>
7      19426
8      19426
9      19426
10    -18625
11    -18625
12    -18625
13    -13053
14    -13053
15    -13053
16     27106
17     27106
18     27106
19     -2590
20     -2590
21     -2590
22    -13053
23    -13053
24    -13053
25     27106
26     27106
27     27106
28     -2590
29     -2590
Name: SampleMaskID, dtype: Int16


In [54]:
# Convert the column to nullable integer type Int32
merged_data_all['SampleMaskID'] = merged_data_all['SampleMaskID'].astype('Int32')
print("memory as Int32: ", merged_data_all['SampleMaskID'].memory_usage(deep=True))

memory as Int32:  14619368


In [55]:
# Display DataFrame info and memory usage
print(merged_data_all.info())
print()
print(merged_data_all.memory_usage(deep=True).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2923848 entries, 0 to 2923847
Columns: 1578 entries, MaskID to LabID
dtypes: Int32(2), Sparse[float16, nan](635), category(941)
memory usage: 2.7 GB
None

2894876745


In [56]:
top_5_values = merged_data_all['SampleMaskID'].value_counts().nlargest(5)
print(top_5_values)

SampleMaskID
2240394    486
5577426    459
4397155    442
5336074    405
3793293    390
Name: count, dtype: Int64


#### turn some category into Sparse[str]

In [57]:
# SexOfRelativeWithT1D14_1 as category in the dataframe with only one value: Male.  
test_col = merged_data_all['SexOfRelativeWithT1D14_1']

print(test_col.memory_usage(deep=True))
print()

test_col_category = test_col.astype('Sparse[str]')
print(test_col_category.memory_usage(deep=True))

2924145

70172521


#### It is an unefficient way to turn string values into sparse even when the column with nearly 100% missing values. 

#### Function to check memory usage of a dataframe

In [58]:
import pandas as pd

# Assuming merged_data_all is your DataFrame

def column_memory_usage(df):
    # List to store memory usage and data types
    memory_info = []

    # Iterating through each column to get memory usage and data type
    for col in df.columns:
        memory = df[col].memory_usage(deep=True)
        dtype = df[col].dtype
        memory_info.append({'Column': col, 'Memory Usage (Bytes)': memory, 'Data Type': dtype})

    # Create a DataFrame from the list
    memory_df = pd.DataFrame(memory_info)

    # Sorting by memory usage in descending order
    memory_df = memory_df.sort_values(by='Memory Usage (Bytes)', ascending=False)

    return memory_df

In [59]:
# Call the function and store the result
column_memory_info = column_memory_usage(merged_data_all)

# Display the result
print(column_memory_info.info())

column_memory_info.to_csv('TN_01_memory_usage_2.csv', index = False)

<class 'pandas.core.frame.DataFrame'>
Index: 1578 entries, 0 to 1511
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Column                1578 non-null   object
 1   Memory Usage (Bytes)  1578 non-null   int64 
 2   Data Type             1578 non-null   object
dtypes: int64(1), object(2)
memory usage: 49.3+ KB
None


## 1.2 TN_16

In [2]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import numpy as np
import warnings
warnings.filterwarnings('ignore')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# The following code is to load 
# load the merged data of TN_16
file_path_16 = "TN_16_merged_data.csv"

TN_16_merged_data = pd.read_csv(file_path_16)

print(TN_16_merged_data.info())
print(TN_16_merged_data.memory_usage(deep=True).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445461 entries, 0 to 2445460
Columns: 262 entries, MaskID to LabID
dtypes: float64(99), object(163)
memory usage: 4.8+ GB
None
19758580453


In [4]:
import pandas as pd
import numpy as np

def transform_data_types(df):
    for column in df.columns:
        # Check for NaN values in the column
        has_nan = df[column].isnull().any()

        if df[column].dtype == 'float64':
            # Convert to int16 if all values are integers and there are no NaN values
            if not has_nan and all(df[column].dropna().apply(float.is_integer)):
                df[column] = df[column].astype('int16')
            else:
                # Convert to float16 otherwise
                df[column] = df[column].astype('float16')

        elif df[column].dtype == 'object':
            # Convert string columns to category
            if all(df[column].apply(lambda x: isinstance(x, str) or pd.isnull(x))):
                df[column] = df[column].astype('category')

    return df

In [ ]:
TN_16_merged_data = transform_data_types(TN_16_merged_data)

print(TN_16_merged_data.info())
print(TN_16_merged_data.memory_usage(deep=True).sum())

In [6]:
summary_df_16 = pd.read_csv('TN_16_merged_data_summary.csv')
summary_df_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          262 non-null    object 
 1   Missing Percentage  262 non-null    float64
 2   Value Type          262 non-null    object 
 3   Top 1               262 non-null    object 
 4   Top 2               212 non-null    object 
 5   Top 3               159 non-null    object 
 6   Top 4               136 non-null    object 
 7   Top 5               121 non-null    object 
 8   Top 1 Value Count   262 non-null    int64  
 9   Top 2 Value Count   212 non-null    float64
 10  Top 3 Value Count   159 non-null    float64
 11  Top 4 Value Count   136 non-null    float64
 12  Top 5 Value Count   121 non-null    float64
dtypes: float64(5), int64(1), object(7)
memory usage: 26.7+ KB


In [9]:
#rename the column 'Unnamed: 0'
summary_df_16.rename(columns={'Unnamed: 0': 'header'}, inplace=True)

In [29]:
# Filter based on Missing Percentage and Value Type
filtered_columns = summary_df_16[(summary_df_16['Missing Percentage'] > 85) & 
                             (summary_df_16['Value Type'].isin(['float64']))]

# Get the list of headers from the filtered DataFrame
tn_16_headers_with_missing_85 = filtered_columns['header'].tolist()

# Print the count of columns
print("Count of columns with missing Percentage > 85% and Value Type 'float64':", len(tn_16_headers_with_missing_85))

Count of columns with missing Percentage > 85% and Value Type 'float64': 60


In [28]:
error_columns = []  # List to store columns that raised errors

for col in tn_16_headers_with_missing_85:
    try:
        print("Before transformation of", col, ":", TN_16_merged_data[col].memory_usage(deep=True), "bytes")
        
        # Check data type of the column
        col_dtype = TN_16_merged_data[col].dtype
        print(col_dtype)
        if col_dtype == 'float16' or col_dtype == 'float64':
            # Convert numeric columns to default sparse type
            TN_16_merged_data[col] = pd.to_numeric(TN_16_merged_data[col], errors='coerce').astype('Sparse[float16]')
        # No need to replace the column in the original DataFrame
        # Just printing out the memory usage after transformation
        print("After transformation to Sparse of", col, ":", TN_16_merged_data[col].memory_usage(deep=True), "bytes")
    except ValueError as e:
        print(f"Error converting column {col}: {e}")
        error_columns.append(col)

    print()

# Print out columns that had errors
if error_columns:
    print("Columns with errors:", error_columns)

Before transformation of AEAssociations_Lifethreatening : 152 bytes
Sparse[float16, nan]
After transformation to Sparse of AEAssociations_Lifethreatening : 152 bytes

Before transformation of BloodPressureNotDone : 752 bytes
Sparse[float16, nan]
After transformation to Sparse of BloodPressureNotDone : 752 bytes

Before transformation of AnyNewChronicDisease_Neurologica : 764 bytes
Sparse[float16, nan]
After transformation to Sparse of AnyNewChronicDisease_Neurologica : 764 bytes

Before transformation of AEAssociations_Isanotherconditio : 2288 bytes
Sparse[float16, nan]
After transformation to Sparse of AEAssociations_Isanotherconditio : 2288 bytes

Before transformation of HaveHadComplications_Other : 5438 bytes
Sparse[float16, nan]
After transformation to Sparse of HaveHadComplications_Other : 5438 bytes

Before transformation of AESelectTerm : 9248 bytes
Sparse[float16, nan]
After transformation to Sparse of AESelectTerm : 9248 bytes

Before transformation of AEDateDeathYear : 9428 

In [23]:
#  
test_col = TN_16_merged_data['LowestGlucosemgdl'] 

print("original memory usage: ", test_col.memory_usage(deep=True))
test_col_32 = test_col.astype('Sparse[float16]')
print("memory usage as Int32: ", test_col_32.memory_usage(deep=True))

original memory usage:  4891050
memory usage as Int32:  4312352


In [30]:
print(TN_16_merged_data.info())
print(TN_16_merged_data.memory_usage(deep=True).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445461 entries, 0 to 2445460
Columns: 262 entries, MaskID to LabID
dtypes: Sparse[float16, nan](75), category(151), float16(24), object(12)
memory usage: 814.6+ MB
None
1614482009


In [60]:
# Call the function and store the result
column_memory_tn_16= column_memory_usage(TN_16_merged_data)

# Display the result
print(column_memory_tn_16.info())

column_memory_tn_16.to_csv('TN_16_memory_usage_0.csv', index = False)

<class 'pandas.core.frame.DataFrame'>
Index: 262 entries, 254 to 20
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Column                262 non-null    object
 1   Memory Usage (Bytes)  262 non-null    int64 
 2   Data Type             262 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.2+ KB
None


#### change MaskID into int32

In [63]:
# Function to check if a value can be converted to a numeric type
def is_not_numeric(val):
    try:
        float(val)  # Try converting to float
        return False
    except ValueError:
        return True

# Apply the function to filter out non-numeric values
MaskID_non_numeric_values = TN_16_merged_data['MaskID'][TN_16_merged_data['MaskID'].apply(is_not_numeric)]
print(MaskID_non_numeric_values)

1220981        3/0.020mg (Drospiren/EthEstrs/.020
1873250                        unknown start date
1879483                                 2000units
2070561    (Does Not Take Over the Summer Months)
2092198                               (1mg/20mcg)
Name: MaskID, dtype: object


In [73]:
# Function to check if a value is not an integer
def is_not_integer(val):
    try:
        float_val = float(val)  # Convert to float first
        int_val = int(float_val)
        return float_val != int_val  # Check if float value is not equal to its integer conversion
    except ValueError:
        return True  # Non-numeric values are also considered as not integer

# Apply the function to filter out non-integer values
MaskID_non_integer_values = TN_16_merged_data['MaskID'][TN_16_merged_data['MaskID'].apply(is_not_integer)]
print(MaskID_non_integer_values)

Series([], Name: MaskID, dtype: object)


In [71]:
print(TN_16_merged_data.info())
TN_16_merged_data = TN_16_merged_data.drop(MaskID_non_integer_values.index)
print(TN_16_merged_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2445456 entries, 0 to 2445460
Columns: 262 entries, MaskID to LabID
dtypes: Sparse[float64, nan](75), category(151), float16(24), object(12)
memory usage: 913.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 2445453 entries, 0 to 2445460
Columns: 262 entries, MaskID to LabID
dtypes: Sparse[float64, nan](75), category(151), float16(24), object(12)
memory usage: 913.8+ MB
None


In [79]:
import pandas as pd

# Assuming TN_16_merged_data is your DataFrame and 'MaskID' is the column you're working with
test_col = TN_16_merged_data['MaskID']

print(test_col.memory_usage(deep=True))
print(test_col.head(30))

# Convert to numeric, coercing errors to NaN
test_col_numeric = pd.to_numeric(test_col, errors='coerce')
test_col_32 = test_col_numeric.astype('Int32')

print(test_col_32.memory_usage(deep=True))
print(test_col_32.head(30))
print()


107868097
0     378529
1     378529
2     378529
3     378529
4     378529
5     378529
6     378529
7     378529
8     378529
9     378529
10    378529
11    378529
12    378529
13    378529
14    378529
15    378529
16    378529
17    378529
18    378529
19    378529
20    378529
21    378529
22    378529
23    378529
24    378529
25    378529
26    378529
27    378529
28    378529
29    378529
Name: MaskID, dtype: object
31790889
0     378529
1     378529
2     378529
3     378529
4     378529
5     378529
6     378529
7     378529
8     378529
9     378529
10    378529
11    378529
12    378529
13    378529
14    378529
15    378529
16    378529
17    378529
18    378529
19    378529
20    378529
21    378529
22    378529
23    378529
24    378529
25    378529
26    378529
27    378529
28    378529
29    378529
Name: MaskID, dtype: Int32



In [84]:
print(TN_16_merged_data.memory_usage(deep=True).sum())

test_col_numeric = pd.to_numeric(TN_16_merged_data['MaskID'], errors='coerce')
TN_16_merged_data['MaskID'] = test_col_numeric.astype('Int32')

print(TN_16_merged_data.memory_usage(deep=True).sum())

1718516099
1642438891


In [85]:
TN_16_merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2445453 entries, 0 to 2445460
Columns: 262 entries, MaskID to LabID
dtypes: Int32(1), Sparse[float64, nan](75), category(151), float16(24), object(11)
memory usage: 906.8+ MB


In [88]:
#  
test_col = TN_16_merged_data['AssessDateYear'] 

print("original memory usage as object: ", test_col.memory_usage(deep=True))
test_col_category = test_col.astype('category')
print("memory usage as category: ", test_col_category.memory_usage(deep=True))

test_col_numeric = pd.to_numeric(test_col, errors='coerce')
test_col_16 = test_col_numeric.astype('Int16')
print("memory usage as int16: ", test_col_16.memory_usage(deep=True))

original memory usage as object:  106806701
memory usage as category:  22009851
memory usage as int16:  26899983


In [90]:
for column in TN_16_merged_data.columns:
    if TN_16_merged_data[column].dtype == 'object':
        print("original memory usage as object: ", TN_16_merged_data[column].memory_usage(deep=True))
        TN_16_merged_data[column] = TN_16_merged_data[column].astype('category')
        print("memory usage as category: ", TN_16_merged_data[column].memory_usage(deep=True))

original memory usage as object:  106806701
memory usage as category:  22009851
original memory usage as object:  106291552
memory usage as category:  22015592
original memory usage as object:  100570045
memory usage as category:  22014662
original memory usage as object:  97939172
memory usage as category:  22009578
original memory usage as object:  97907624
memory usage as category:  22010525
original memory usage as object:  97900952
memory usage as category:  22009821
original memory usage as object:  97944312
memory usage as category:  22009819
original memory usage as object:  97838284
memory usage as category:  22009789
original memory usage as object:  97854668
memory usage as category:  22009757
original memory usage as object:  98633864
memory usage as category:  22009465
original memory usage as object:  119392425
memory usage as category:  24756402


In [91]:
print(TN_16_merged_data.memory_usage(deep=True).sum())
TN_16_merged_data.info()

768224553
<class 'pandas.core.frame.DataFrame'>
Index: 2445453 entries, 0 to 2445460
Columns: 262 entries, MaskID to LabID
dtypes: Int32(1), Sparse[float64, nan](75), category(162), float16(24)
memory usage: 729.7 MB


#### Turn Sparse[float64] into Sparse[float16]

In [95]:
# Iterate over each column and convert if it's of type Sparse[float64]
for column in TN_16_merged_data.columns:
    if pd.api.types.is_sparse(TN_16_merged_data[column].dtype):
        if TN_16_merged_data[column].dtype.subtype == np.float64:
            TN_16_merged_data[column] = TN_16_merged_data[column].astype('Sparse[float16]')

print(TN_16_merged_data.info())
print()
print(TN_16_merged_data.memory_usage(deep=True).sum())

<class 'pandas.core.frame.DataFrame'>
Index: 2445453 entries, 0 to 2445460
Columns: 262 entries, MaskID to LabID
dtypes: Int32(1), Sparse[float16, nan](75), category(162), float16(24)
memory usage: 649.2 MB
None

683748849


In [93]:
shared_columns = TN_16_merged_data.columns.intersection(merged_data_all.columns)
print(len(shared_columns))
print(shared_columns)

79
Index(['MaskID', 'Visit_Dt', 'Visit', 'AdverseEventID', 'AETerm', 'severity',
       'AEReportType', 'AEReportDtDay', 'AEReportDtMonth', 'AEReportDtYear',
       'AEOccurDtDay', 'AEOccurDtMonth', 'AEOccurDtYear',
       'AEPrimarySecondary1_1', 'AECategory', 'AETreatLocation',
       'AEExpected1_1', 'AECausalityByReporter', 'AEAssociations_Death',
       'AEAssociations_Hospitalizationor', 'AEAssociations_Lifethreatening',
       'AEAssociations_Isanotherconditio', 'AEPatientOutcome1_1',
       'AEResolveDtDay', 'AEResolveDtMonth', 'AEResolveDtYear',
       'AEDateDeathDay', 'AEDateDeathMonth', 'AEDateDeathYear',
       'Eventabateafterstoppingdrug', 'Eventreappearafterreintroducti',
       'AEFollowupDateDay', 'AEFollowupDateMonth', 'AEFollowupDateYear',
       'AEReasonForFollowup', 'AEFollowupNumber', 'AEReporterStaffCode',
       'AESelectTerm', 'AEReviewTreatmentUnblindedForC',
       'AECausalityByReviewer', 'AEReviewPriorSimilarSAEs',
       'AEReviewReportingTimeframe', 'Da

In [98]:
# Check for inconsistencies in data types and only report those
inconsistencies = {}
for col in shared_columns:
    dtype_df1 = TN_16_merged_data[col].dtype
    dtype_df2 = merged_data_all[col].dtype

    if dtype_df1 != dtype_df2:
        print(f"Column '{col}' has inconsistent data types: {dtype_df1} (DataFrame 1) != {dtype_df2} (DataFrame 2)")

# Print only the columns with inconsistent data types
for col, dtypes in inconsistencies.items():
    print(f"Column '{col}' has inconsistent data types: {dtypes[0]} (DataFrame 1) vs {dtypes[1]} (DataFrame 2)")

Column 'Visit_Dt' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'Visit' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'AETerm' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'AEPrimarySecondary1_1' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'AECategory' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'AETreatLocation' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'AECausalityByReporter' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'AEPatientOutcome1_1' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'Eventabateafterstoppingdrug' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'Eventreappearafterreintroducti' has inconsistent data types: categor

In [96]:
# Check for inconsistencies in data types
for col in shared_columns:
    dtype_df1 = TN_16_merged_data[col].dtype
    dtype_df2 = merged_data_all[col].dtype

    if dtype_df1 != dtype_df2:
        print(f"Column '{col}' has inconsistent data types: {dtype_df1} (DataFrame 1) != {dtype_df2} (DataFrame 2)")
    else:
        print(f"Column '{col}' has consistent data type: {dtype_df1}")

Column 'MaskID' has consistent data type: Int32
Column 'Visit_Dt' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'Visit' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'AdverseEventID' has consistent data type: Sparse[float16, nan]
Column 'AETerm' has inconsistent data types: category (DataFrame 1) != category (DataFrame 2)
Column 'severity' has consistent data type: Sparse[float16, nan]
Column 'AEReportType' has consistent data type: category
Column 'AEReportDtDay' has consistent data type: Sparse[float16, nan]
Column 'AEReportDtMonth' has consistent data type: Sparse[float16, nan]
Column 'AEReportDtYear' has consistent data type: Sparse[float16, nan]
Column 'AEOccurDtDay' has consistent data type: Sparse[float16, nan]
Column 'AEOccurDtMonth' has consistent data type: Sparse[float16, nan]
Column 'AEOccurDtYear' has consistent data type: Sparse[float16, nan]
Column 'AEPrimarySecondary1_1' has inconsistent data 

In [ ]:
common_headers_123 = set.intersection(set(transformed_merged_first_part.columns), set(transformed_merged_second_part.columns))

# Merge the first two DataFrames on common headers using outer join
merged_data_all = pd.merge(transformed_merged_first_part, transformed_merged_second_part, on=list(common_headers_123), how='outer', 
                            suffixes=('_dup_x', '_dup_y'))

print(merged_data_all.info())
merged_data_all.head()

## 1.3 TN_19

## 1.4 TN_20

#### test of merged dataset

In [2]:
string = "D5OtherTestResultDD"  # change this to the string you're searching for
# Convert the search string to lower case (or upper case)
search_string_lower = string.lower()

# Find columns that contain the search string, case-insensitive
matching_columns = [col for col in transformed_merged_data.columns if search_string_lower in col.lower()]
print("Found ", len(matching_columns), "columns: ", matching_columns)

# Create a DataFrame with the filtered columns and drop rows with only missing values
filtered_df = transformed_merged_data[matching_columns].dropna(how='all')
print("DataFrame with filtered columns (rows with all missing values dropped):")
print()
print(filtered_df.info())
#print(filtered_df.head(30))

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(transformed_merged_data[col].value_counts().nlargest(30))
    print()

NameError: name 'transformed_merged_data' is not defined

In [34]:
summary_df = pd.read_csv('merged_first_part_summary.csv')

In [12]:
merged_data_23.memory_usage(deep=True)

Index                                  128
MaskID                            15750456
Date_of_Registration              89150884
Status                            88100924
Date_of_Study_Start               88252854
Date_of_Draw                     123874629
Event_Title                      158197379
Spec_Name                        146762318
SampleMaskID                     120396183
Test_Name                        115208219
Result                           109581057
Result_Type                      113438563
Visit                            124265369
Date_at_Test_Results_Reported    123840259
Date_at_Evaluation               123874419
Date_Received                    121761644
Date_Shipped                     123872389
LabID                             17719263
dtype: int64

In [14]:
merged_data_23['Date_of_Registration'] = pd.to_datetime(merged_data_23['Date_of_Registration'], format='%m/%d/%Y')

In [15]:
merged_data_23.memory_usage(deep=True)

Index                                  128
MaskID                            15750456
Date_of_Registration              15750456
Status                            88100924
Date_of_Study_Start               88252854
Date_of_Draw                     123874629
Event_Title                      158197379
Spec_Name                        146762318
SampleMaskID                     120396183
Test_Name                        115208219
Result                           109581057
Result_Type                      113438563
Visit                            124265369
Date_at_Test_Results_Reported    123840259
Date_at_Evaluation               123874419
Date_Received                    121761644
Date_Shipped                     123872389
LabID                             15750456
dtype: int64

In [17]:
merged_data_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968807 entries, 0 to 1968806
Data columns (total 17 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   MaskID                         int64         
 1   Date_of_Registration           datetime64[ns]
 2   Status                         object        
 3   Date_of_Study_Start            object        
 4   Date_of_Draw                   object        
 5   Event_Title                    object        
 6   Spec_Name                      object        
 7   SampleMaskID                   object        
 8   Test_Name                      object        
 9   Result                         object        
 10  Result_Type                    object        
 11  Visit                          object        
 12  Date_at_Test_Results_Reported  object        
 13  Date_at_Evaluation             object        
 14  Date_Received                  object        
 15  Date_Shipped   

In [15]:
# Check unique value counts. 
unique_counts = merged_data_23.nunique()
unique_counts

MaskID                           237048
Date_of_Registration               5740
Status                                5
Date_of_Study_Start                5863
Date_of_Draw                       5786
Event_Title                          49
Spec_Name                            22
SampleMaskID                     745974
Test_Name                            37
Result                            10241
Result_Type                           3
Visit                                50
Date_at_Test_Results_Reported      4451
Date_at_Evaluation                 4350
Date_Received                      4162
Date_Shipped                       4673
LabID                                32
dtype: int64

# 1.4 Data query

## 1.4.1 check column names and their values containing a string

In [2]:
# The following code is to load the first part of merged dataset
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import warnings
warnings.filterwarnings('ignore')

merged_first_part = pd.read_csv('TN_01_merged_data_1.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Strings in columns that could be merged:
1. testName: 23
2. 
3. D5OtherTestResultDD, D5OtherTestResultMM, D5OtherTestResultYYYY: 3*23
4. 
5. 
6. 
7. 
8. Heightcm, heightin
9. Weightkg, Weightlbs
10.'TimeOfCollectionHH', 'TimeOfCollectionMM'
11.
12.
13. 

In [2]:
merged_first_part.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519673 entries, 0 to 519672
Columns: 1563 entries, Unnamed: 0 to SignedConsentFor2017Protocol
dtypes: float64(664), int64(2), object(897)
memory usage: 6.1+ GB


In [25]:
merged_first_part.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519673 entries, 0 to 519672
Columns: 1563 entries, Unnamed: 0 to SignedConsentFor2017Protocol
dtypes: category(136), float64(664), int64(2), object(761)
memory usage: 5.6+ GB


#### 1) Case insensitive

In [ ]:
string = "D6OtherTestResultUnit"  # change this to the string you're searching for
# Convert the search string to lower case (or upper case)
search_string_lower = string.lower()

# Find columns that contain the search string, case-insensitive
matching_columns = [col for col in merged_first_part.columns if search_string_lower in col.lower()]
print("Found ", len(matching_columns), "columns: ", matching_columns)

# Create a DataFrame with the filtered columns and drop rows with only missing values
filtered_df = merged_first_part[matching_columns].dropna(how='all')
print("DataFrame with filtered columns (rows with all missing values dropped):")
print()
print(filtered_df.info())
#print(filtered_df.head(30))

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_first_part[col].value_counts().nlargest(30))
    print()
    

#### 2) Case sensitive

In [ ]:
string = "D5OtherTestResult"  # Change this to the string you're searching for

# Find columns that contain the search string, case-sensitive
matching_columns = [col for col in merged_first_part.columns if string in col]
print("Found ", len(matching_columns), "columns: ", matching_columns)

# Create a DataFrame with the filtered columns and drop rows with only missing values
filtered_df = merged_first_part[matching_columns].dropna(how='all')
print("DataFrame with filtered columns (rows with all missing values dropped):")
print(filtered_df.info())
#print(filtered_df.head(30))

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_first_part[col].value_counts().nlargest(20))
    print()


## 1.4.2 check column names and their values containing two strings

Filtered columns:
1. 

In [4]:
# Strings to search for: AND
string1 = ""  # First string
string2 = "heightin"  # Second string, change this to your second string

# Convert the search strings to lower case
search_string1_lower = string1.lower()
search_string2_lower = string2.lower()

# Find columns that contain both search strings, case-insensitive
matching_columns = [col for col in merged_first_part.columns 
                    if search_string1_lower in col.lower() and search_string2_lower in col.lower()]

print("Found ", len(matching_columns), "columns: ", matching_columns)

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_first_part[col].value_counts().nlargest(20))
    print()


Found  0 columns:  []


In [ ]:
# Strings to search for: OR
string1 = "heightcm"  # First string
string2 = "heightin"  # Second string, change this to your second string

# Convert the search strings to lower case
search_string1_lower = string1.lower()
search_string2_lower = string2.lower()

# Find columns that contain both search strings, case-insensitive
matching_columns = [col for col in merged_first_part.columns 
                    if search_string1_lower in col.lower() or search_string2_lower in col.lower()]

print("Found ", len(matching_columns), "columns: ", matching_columns)

# Create a DataFrame with the filtered columns and drop rows with only missing values
filtered_df = merged_first_part[matching_columns].dropna(how='all')

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_first_part[col].value_counts().nlargest(20))
    print()

In [7]:
filtered_df.describe()

,Heightcm,Heightin,Heightcm_dup14,Heightin_dup14,Heightcm_dup15,Heightin_dup15,Heightcm_dup20,Heightin_dup20,Heightcm_dup21,Heightin_dup21
count,6322.000000,777.000000,470.000000,22.000000,17895.000000,1670.000000,98.000000,26.000000,150.000000,4.000000
mean,145.704470,58.872081,161.463000,59.227273,155.012744,61.237182,166.091327,65.520385,152.980333,59.100000
std,27.089781,11.184336,19.753296,9.314379,25.386499,11.930460,15.612072,6.933721,20.112152,4.313545
min,0.000000,0.000000,39.400000,40.900000,0.000000,0.000000,114.000000,45.000000,97.200000,53.000000
25%,126.500000,51.000000,152.650000,53.035000,141.000000,56.925000,160.225000,63.075000,138.485000,57.875000
50%,151.000000,62.100000,164.700000,61.000000,160.500000,64.000000,167.950000,65.750000,158.200000,60.200000
75%,167.000000,67.000000,175.500000,66.400000,170.700000,68.000000,176.775000,70.922500,167.002500,61.425000
max,219.000000,77.000000,196.170000,72.000000,1658.500000,185.000000,196.100000,75.000000,194.800000,63.000000


## 1.4.3 check column headers in one shared CSV file

In [40]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED

#file_path = ".././studies/TN_01/TrialNet_01_Data/TN01_ADVERSEEVENTS.CSV"
file_path = "/home/ec2-user/SageMaker/studies/TN_01/TrialNet_01_Data/TN01_nh08_diabonset_current.csv"

df1 = pd.read_csv(file_path)
df1.head()

,MaskID,Visit_Dt,Visit,CriteriaParticipantDiagnosewit,D1FastingPlasmaGlucDD1_1,D1FastingPlasmaGlucMM1_1,D1FastingPlasmaGlucYYYY1_1,D1FastingPlasmaGlucResult1_1,D1FastingPlasmaGlucUnits1_1,D1FastingPlasmaGlucDD2_1,...,D5OtherTestResultYYYY14_1,D5OtherTestResultYYYY15_1,D5OtherTestResultYYYY16_1,D5OtherTestResultYYYY17_1,D5OtherTestResultYYYY18_1,D5OtherTestResultYYYY19_1,D5OtherTestResultYYYY20_1,D5OtherTestResultYYYY21_1,D5OtherTestResultYYYY22_1,D5OtherTestResultYYYY23_1
0,202411,11/06/2018,PRN,b.DKA or unequivocally symptomatic AND one dia...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202455,10/08/2019,PRN,e.Diagnosis made by criteria other than listed...,16.0,Sep,2019.0,11.6,mmol/L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,203313,05/19/2020,PRN,e.Diagnosis made by criteria other than listed...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,203899,09/15/2020,PRN,b.DKA or unequivocally symptomatic AND one dia...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,207308,02/04/2019,PRN,a.Two consecutive OGTT clinical alerts,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
string = "MM"  # change this to the string you're searching for
# Convert the search string to lower case (or upper case)
search_string_lower = string.lower()

# Find columns that contain the search string, case-insensitive
matching_columns = [col for col in df1.columns if search_string_lower in col.lower()]
print("Found ", len(matching_columns), "columns: ", matching_columns)


matching_columns2 = ['MaskID', 'Visit_Dt'] + matching_columns
# Create a DataFrame with the filtered columns and drop rows with only missing values
filtered_df = df1[matching_columns2].dropna(how='all', subset=matching_columns)
print("DataFrame with filtered columns (rows with all missing values dropped):")
#print(filtered_df.info())
#print(filtered_df.head(30))

# Print top 20 values for each matching column
for col in matching_columns:
    print(f"Top 20 values for column {col}:")
    print(df1[col].value_counts().nlargest(20))
    print()


In [13]:
# Display the selected rows
filtered_df.head(20)

,MaskID,Visit_Dt,D5MeterReadingLoc1_1,D5MeterReadingDD1_1,D5MeterReadingMM1_1,D5MeterReadingYYYY1_1,D5MeterReadingResult1_1,D5MeterReadingUnit1_1,D5MeterReadingLoc2_1,D5MeterReadingDD2_1,...,D5MeterReadingMM7_1,D5MeterReadingDD5_1,D5MeterReadingDD6_1,D5MeterReadingDD7_1,D5MeterReadingResult5_1,D5MeterReadingResult6_1,D5MeterReadingResult7_1,D5MeterReadingYYYY5_1,D5MeterReadingYYYY6_1,D5MeterReadingYYYY7_1
0,202411,11/06/2018,In Home,26.0,Sep,2018.0,160.0,mg/dl,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202455,10/08/2019,In Clinic,18.0,Sep,2019.0,12.5,mmol/L,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,203313,05/19/2020,In Home,23.0,Apr,2020.0,22.0,mmol/L,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,216884,07/29/2019,In Clinic,23.0,Jul,2019.0,325.0,mg/dl,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,217158,04/16/2019,In Clinic,26.0,Mar,2019.0,90.0,mg/dl,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,218512,10/26/2020,In Clinic,17.0,Feb,2020.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,231753,11/25/2019,In Home,27.0,Apr,2018.0,138.0,mg/dl,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,242474,07/26/2018,In Home,22.0,Aug,2017.0,307.0,mg/dl,In Clinic,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,247747,04/25/2019,In Home,24.0,Apr,2019.0,13.9,mmol/L,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,248809,10/13/2020,In Home,23.0,Jun,2020.0,7.4,mmol/L,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Save the DataFrame to a CSV file
#filtered_df.to_csv('D5MeterReading_TN01_nh08_diabonset_current.csv')

In [6]:
# Filter the dataframe based on the condition
filtered_rows = filtered_df[filtered_df['D5MeterReadingLoc1_1'] == 'Unknown']

# Display the first 20 rows of the filtered dataframe
filtered_rows.head(20)


,MaskID,Visit_Dt,D5MeterReadingLoc1_1,D5MeterReadingDD1_1,D5MeterReadingMM1_1,D5MeterReadingYYYY1_1,D5MeterReadingResult1_1,D5MeterReadingUnit1_1,D5MeterReadingLoc2_1,D5MeterReadingDD2_1,...,D5MeterReadingMM7_1,D5MeterReadingDD5_1,D5MeterReadingDD6_1,D5MeterReadingDD7_1,D5MeterReadingResult5_1,D5MeterReadingResult6_1,D5MeterReadingResult7_1,D5MeterReadingYYYY5_1,D5MeterReadingYYYY6_1,D5MeterReadingYYYY7_1
32,260645,10/29/2020,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,467630,01/14/2020,Unknown,15.0,Nov,2019.0,17.6,mmol/L,Unknown,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,518555,06/19/2019,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,529847,01/24/2020,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,567236,07/18/2018,Unknown,5.0,Jul,2018.0,681.0,mg/dl,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,600180,08/14/2019,Unknown,1.0,Aug,2019.0,234.0,mg/dl,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,698893,02/10/2020,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
